In [1]:
import os 
import sys
base_dir = os.path.abspath('')
curr_dir = os.path.dirname(base_dir)
if not curr_dir in sys.path: sys.path.append(curr_dir)

In [2]:
from glob import glob
from utils import to_markdown
from urllib.parse import urlparse
from collections import defaultdict

In [3]:
def contains_course_info(url):
    url_prased = urlparse(url)
    return len(url_prased.path.split("/"))>5

In [4]:
files = []
with open("../stevens_scraper/crawled_urls.txt") as fp:
    for line in fp:
        files.append(line.strip())
files = list(map(lambda x: x.lower(), files))
files = list(set(files))
files = list(filter(lambda file: "narrative-courses" not in file, files))
files = list(filter(lambda file: "javascript" not in file, files))
files = list(filter(lambda file: ".pdf" not in file, files))
files = list(filter(lambda file: ".aspx" not in file, files))
files = list(filter(lambda file: "degree-programs2" not in file, files))
files = list(filter(lambda file: "2023-2024" in file, files))
files = list(filter(lambda file: "#middle" not in file, files))
files = list(filter(contains_course_info , files))
print(len(files))

with open("../stevens_scraper/crawled_urls_v1.txt", "w+") as fp:
    fp.write("\n".join(sorted(files, key=len)))

3434


In [5]:
primary = defaultdict(list)
for url in files:
    url_prased = urlparse(url.lower())
    cat = url_prased.path.replace("/en/2023-2024/academic-catalog/", "").split("/")[0]
    primary[cat].append(url)

In [6]:
for key in primary.keys(): print(key, len(primary[key]))

courses 2889
department-of-civil-environmental-and-ocean-engineering 29
school-of-systems-and-enterprises 41
undergraduate-education 77
department-of-computer-science 23
school-of-humanities-arts-and-social-sciences 28
 12
school-of-business 73
tuition-fees-and-other-expenses-for-graduate-students 5
graduate-education 24
department-of-electrical-and-computer-engineering 35
student-services 26
department-of-physics 17
department-of-mechanical-engineering 31
financing-a-stevens-education 44
student-life 6
department-of-biomedical-engineering 11
department-of-chemical-engineering-and-materials-science 22
tuition-fees-and-other-expenses-for-undergraduate-students 5
department-of-chemistry-and-chemical-biology 16
department-of-mathematical-sciences 14
academic-integrity 2
schaefer-school-of-engineering-and-science 4


In [7]:
sorted(primary["courses"])

['https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/500',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/500/aai-551',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600/aai-627',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600/aai-628',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600/aai-646',
 'https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/courses/aai-applied-artificial-intelligence/600/aai-672',
 'https://stevens.smartcatal

In [8]:
def create_department_school_metadata(url):
    metadata = {}
    path = urlparse(url).path.replace("/en/2023-2024/academic-catalog/", "")
    if "faculty" in path:
        metadata["content_type"] = "Faculty"
    elif "graduate-certificate" in path:
        metadata["content_type"] = "Certificate"
        metadata["program_type"] = "Certificate"
    elif "graduate" in path:
        metadata["content_type"] = "Graduate"
        if "master" in path:
            metadata["program_type"] = "Masters"
        elif "doctoral" in path:
            metadata["program_type"] = "PhD"
        elif "engineer" in path:
            metadata["program_type"] = "Engineer"
    elif "undergraduate" in path:
        metadata["content_type"] = "Undergraduate"
        metadata["program_type"] = "Bachelors"
    path = path.split("/")
    if len(path)>1:
        program_name = path[-1].replace("-", " ").title()
        program_name = program_name.replace("Copy Of", "").strip()
        program_name = program_name.replace("Master Of Science In", "").strip()
        program_name = program_name.replace("Bachelor Of Science In", "").strip()
        program_name = program_name.replace("Doctoral Program In", "").strip()
        program_name = program_name.replace("Engineering Degree In", "").strip()
        metadata["program_name"] = program_name
    return metadata

In [9]:
create_department_school_metadata(primary["department-of-computer-science"][0])

{'content_type': 'Certificate',
 'program_type': 'Certificate',
 'program_name': 'Cybersecurity'}

In [10]:
school_department_keys = [
    "school-of-business",
    "department-of-chemical-engineering-and-materials-science",
    "department-of-electrical-and-computer-engineering",
    "department-of-mathematical-sciences",
    "department-of-chemistry-and-chemical-biology",
    "department-of-computer-science",
    "department-of-civil-environmental-and-ocean-engineering",
    "department-of-mechanical-engineering",
    "department-of-physics",
    "school-of-systems-and-enterprises",
    "schaefer-school-of-engineering-and-science",
    "department-of-biomedical-engineering",
]

In [11]:
url_meta = {}
for key in primary.keys():
    if key in school_department_keys:
        for url in primary[key]:
            meta_data = create_department_school_metadata(url)
            meta_data["department"] = key.replace("-", " ").title()
            url_meta[url] = meta_data

In [12]:
len(url_meta.keys())

316

In [13]:
url_meta["https://stevens.smartcatalogiq.com/en/2023-2024/academic-catalog/department-of-computer-science/graduate-programs/copy-of-master-of-science-in-computer-science"]

{'content_type': 'Graduate',
 'program_type': 'Masters',
 'program_name': 'Computer Science',
 'department': 'Department Of Computer Science'}

In [14]:
import pandas as pd
df = pd.read_csv("../data/department_info.csv")

In [15]:
assert df.shape[0]==df["url"].str.endswith("/").sum()

In [16]:
df["url"] = df["url"].apply(lambda x: x[:-1])

In [17]:
df = df.set_index("url")

In [18]:
department_info = df.to_dict(orient="index")

In [19]:
department_info[list(url_meta.keys())[2]]

{'text': '\n\n\r\n\t\tAdvanced Certificate for Executives in Construction Management\r\n\t\n\n\r\n\t\t\tAdvanced Certificate for Executives in Construction Management Requirements\r\n\t\t\r\n\t\t\tRequired Courses:\r\n\t\t\n\nCM 560Sustainable Design3\n\nCM 530Strategic Responses to Cyclical Environments3\n\nCM 550Construction Contract Law I3\n\nAnd one (1) additional elective course chosen in conjunction with your Faculty Advisor.\n\n\n',
 'html': '\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" lang="en">\r\n<head><title>\r\n\tStevens Institute of TechnologyÂ\xa0-Â\xa0Advanced Certificate for Executives in Construction Management\r\n</title><meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" /><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"beacon"

In [20]:
for url in url_meta:
    if department_info.get(url):
        url_meta[url].update({"page_content": department_info[url]["text"]})

In [21]:
url_meta_complete = []
for url, meta in url_meta.items():
    if meta.get("page_content"):
        # meta.update({"url": url})
        url_meta_complete.append(meta)

In [22]:
url_meta_complete[1]

{'content_type': 'Certificate',
 'program_type': 'Certificate',
 'program_name': 'Environmental Processes',
 'department': 'Department Of Civil Environmental And Ocean Engineering',
 'page_content': '\n\n\r\n\t\tEnvironmental Processes\r\n\t\n\n\r\n\t\t\tEnvironmental Processes Requirements\r\n\t\t\r\n\t\t\tRequired Courses:\r\n\t\t\n\nEN 541Fate and Transport of Environmental Contaminants3\n\nEN 570Environmental Chemistry3\n\nEN 571Physicochemical Processes for Environmental Control3\n\nEN 573Biological Processes for Environmental Control3\n\n\n\n\n'}

In [24]:
pd.DataFrame.from_records(url_meta_complete).to_csv("../data/department_info_w_meta.csv", index=False)

In [23]:
# for url in url_meta:
#     with open("stevens_scraper/department_urls.txt", "a") as fp:
#         fp.write(url+"\n")

In [24]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.retrievers.self_query.chroma import ChromaTranslator

In [25]:
model_kwargs = {
    "trust_remote_code": True,
    # "device": "cpu"
    }
encode_kwargs={"normalize_embeddings": True}
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [28]:
docs = [
    Document(
        page_content=doc["page_content"],
        metadata=dict((key, value) for key, value in doc.items() if key!="page_content")
    )
    for doc in url_meta_complete
]
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(docs, embeddings, collection_name="department_info", persist_directory="../data/chromadb")
# vectorstore = Chroma(collection_name="department_info", persist_directory="../data/chromadb", embedding_function=embeddings)

In [29]:
vectorstore._collection.count()

306

In [30]:
vectorstore.persist()

c:\Users\smend\miniconda3\envs\stevens\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [30]:
        # Ensure to only use the contain comparator while querying this field.

In [31]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="content_type",
        description="""
        The type of the content for each department 
        Can contain the following options individually or together [Faculty, Certificate, Graduate, Undergraduate]
        Ensure to do a case insensitive match
        """.strip(),
        type="string",
    ),
    AttributeInfo(
        name="program_name",
        description="""
        The name of the Program
        Sample program names are Compute Science and Machine Learning
        Ensure to do a case insensitive match
        """.strip(),
        type="string",
    ),
    AttributeInfo(
        name="department",
        description="The department or school that has that program",
        type="string",
    ),
    AttributeInfo(
        name="program_type",
        description="""
        The type of the program offered by the university ['Masters', 'Bachelors', 'Certificate', 'PhD', 'Engineer']
        MS/Master stands for Masters
        BS/Bachelor stands for Bachelors
        Doctoral/Doctrate stands for PhD
        Engg stands for Engineer 
        Ensure to do a case insensitive match
        """.strip(),
        type="string",
    ),
]
document_content_description = "URL of the content"
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    structured_query_translator=ChromaTranslator(),
    enable_limit=True
)

In [32]:
question = """What are the elective courses for MS in Machine Learning?""".strip()

In [33]:
retriever_prompt = f"""
Pick the top 5 result for the following question: ```{question}```
""".strip()

In [34]:
# from langchain.chains.query_constructor.base import (
#     StructuredQueryOutputParser,
#     get_query_constructor_prompt,
# )
# prompt = get_query_constructor_prompt(
#     document_content_description,
#     metadata_field_info,
# )

# output_parser = StructuredQueryOutputParser.from_components()
# query_constructor = prompt | llm | output_parser

# print(prompt.format(query=retriever_prompt))

In [35]:
# print(query_constructor.invoke(question))

In [36]:
# context = retriever.invoke(retriever_prompt)
# context

In [37]:
# context[0].metadata

In [38]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [   
        ("system", "You are an assistant for question-answering tasks related to Stevens Institute Of Technology."),
        ("human", """ 
        Use the following pieces of retrieved context to answer the question.
        If you don't know the answer, just say that you don't know.
        If the topic is related to a course then ensure to mention to course numbers and display the result as a table.
        Answer in markdown format and render tables without code
        Question: {question}
        Context: {context}
        Answer:"""),
    ]
)
chat_template

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assistant for question-answering tasks related to Stevens Institute Of Technology.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template=" \n        Use the following pieces of retrieved context to answer the question.\n        If you don't know the answer, just say that you don't know.\n        If the topic is related to a course then ensure to mention to course numbers and display the result as a table.\n        Answer in markdown format and render tables without code\n        Question: {question}\n        Context: {context}\n        Answer:"))])

In [39]:
combine_docs_chain = create_stuff_documents_chain(llm, chat_template)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [40]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | SelfQueryRetriever(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B280370310>, query_constructor=RunnableBinding(bound=FewShotPromptTemplate(input_variables=['query'], examples=[{'i': 1, 'data_source': '```json\n{{\n    "content": "Lyrics of a song",\n    "attributes": {{\n        "artist": {{\n            "type": "string",\n            "description": "Name of the song artist"\n        }},\n        "length": {{\n            "type": "integer",\n            "description": "Length of the song in seconds"\n        }},\n        "genre": {{\n            "type": "string",\n            "description": "The song genre, one of "pop", "rock" or "rap""\n        }}\n    }}\n}}\n```', 'user_query': 'What are songs by Taylor Swift or Katy Perry about teenage romance under 3 minutes long in the dance pop genre', 'structured_request': '```json\n{

In [41]:
response = retrieval_chain.invoke({"input": retriever_prompt, "question": question})
to_markdown(response["answer"])

> | Machine Learning Core Electives |
> |---------------------------------|
> | CS 513 - Knowledge Discovery and Data Mining |
> | CS 532 - 3D Computer Vision |
> | CS 544 - Health Informatics |
> | CS 556 - Mathematical Foundations of Machine Learning |
> | CS 558 - Computer Vision |
> | CS 582 - Causal Inference |
> | CS 589 - Text Mining and Information Retrieval |
> | CS 598 - Visual Information Retrieval |
> | CS 609 - Data Management and Exploration on the Web |
> | BIA 654 - Experimental Design II |
> | BIA 660 - Web Mining |
> | BIA 662 - Augmented Intelligence and Generative AI |
> | BIA 678 - Big Data Technologies |
> | CPE 608 - Applied Modeling and Optimization |
> | CPE 695 - Applied Machine Learning |
> | FE 541 - Applied Statistics with Applications in Finance |
> | MA 541 - Statistical Methods |
> | MA 630 - Advanced Optimization Methods |
> | MA 641 - Time Series Analysis I |
> | MA 661 - Dynamic Programming and Reinforcement Learning |
> | CS 800 - Special Problems in Computer Science (M.S.) |
> | CS 900 - Thesis in Computer Science (M.S.) |

In [71]:
vectorstore.delete_collection()

In [74]:
for key in primary.keys(): 
    if key not in school_department_keys:
        print(key, len(primary[key])) 

courses 2889
undergraduate-education 77
financing-a-stevens-education 44
student-services 26
 12
tuition-fees-and-other-expenses-for-undergraduate-students 5
graduate-education 24
school-of-humanities-arts-and-social-sciences 28
student-life 6
academic-integrity 2
tuition-fees-and-other-expenses-for-graduate-students 5
